In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import sagemaker.amazon.common as smac
import boto3
import os

In [2]:
dirname = "/Users/dhrebenach/Documents/AICamp_AWS/MusicGenreRecognition"
raw_data_file = os.path.join(dirname, 'data/raw/features_30_sec.csv')
processed_subdir = "standardized"

train_features_file = os.path.join(raw_data_file, processed_subdir, "train/csv/features.csv")
train_labels_file = os.path.join(raw_data_file, processed_subdir, "train/csv/labels.csv")
test_features_file = os.path.join(raw_data_file, processed_subdir, "test/csv/features.csv")
test_labels_file = os.path.join(raw_data_file, processed_subdir, "test/csv/labels.csv")

# read raw data
cols = list(pd.read_csv(raw_data_file, nrows =1))
raw = pd.read_csv(raw_data_file, sep=',', index_col ='filename', header=0, usecols=[i for i in cols if i != 'label'])
labels = pd.read_csv(raw_data_file, usecols=[59])

# split into train/test with a 90/10 split
train_features, test_features, train_labels, test_labels = train_test_split(raw, labels, test_size=0.1, random_state=42)

# features/labels are returned from train_test_split func in pandas DataFrame format
# convert these to numpy arrays
train_features_arr = train_features.to_numpy()
train_labels_arr = train_labels.to_numpy()
test_features_arr = test_features.to_numpy()
test_labels_arr = test_labels.to_numpy()

# labels are currently in the form of strings, representing music genres
# covert these strings to their float64 int equivalents
train_labels_ints = np.array([])
for x in range (0, len(train_labels_arr)):
    bytes = train_labels_arr[x][0].encode("utf-8")
    int_label = int.from_bytes(bytes, byteorder="big")
    int_label_to_float64 = np.float64(int_label)
    train_labels_ints = np.append(train_labels_ints, int_label_to_float64)

test_labels_ints = np.array([])
for x in range (0, len(test_labels_arr)):
    bytes = test_labels_arr[x][0].encode("utf-8")
    int_label = int.from_bytes(bytes, byteorder="big")
    int_label_to_float64 = np.float64(int_label)
    test_labels_ints = np.append(test_labels_ints, int_label_to_float64)
    
num_features = raw.shape[1]
print("num_features: ", num_features)
raw.head()

num_features:  58


,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
filename,,,,,,,,,,,,,,,,,,,,,
blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,...,0.752740,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035
blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,...,0.927998,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282
blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,...,2.451690,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025
blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,...,0.780874,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339
blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,...,-4.520576,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160


In [3]:
print('train_features shape = ', train_features.shape)
print('train_labels shape = ', train_labels_ints.shape)

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, train_features_arr, train_labels_ints)
buf.seek(0)

train_features shape =  (900, 58)
train_labels shape =  (900,)


0

In [4]:
bucket = 'music-genre-recognition'
prefix = 'music-genre-recognition'
key = 'gtzan-data'

boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

uploaded training data location: s3://music-genre-recognition/music-genre-recognition/train/gtzan-data


In [5]:
print('test_features shape = ', test_features_arr.shape)
print('test_labels shape = ', test_labels_ints.shape)

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, test_features_arr, test_labels_ints)
buf.seek(0)

boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test', key)).upload_fileobj(buf)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, prefix, key)
print('uploaded test data location: {}'.format(s3_test_data))

test_features shape =  (100, 58)
test_labels shape =  (100,)
uploaded test data location: s3://music-genre-recognition/music-genre-recognition/test/gtzan-data
